In [ ]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

import thicket as th
import os

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [ ]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tkSorted = th.Thicket.from_caliperreader(glob("caliper_files/Sorted/*.cali"));
tkRandom = th.Thicket.from_caliperreader(glob("caliper_files/Random/*.cali"));
tkReverse = th.Thicket.from_caliperreader(glob("caliper_files/ReverseSorted/*.cali"));
tkPerturbed = th.Thicket.from_caliperreader(glob("caliper_files/1_perc_perturbed/*.cali"));

In [ ]:
# print(tkSorted.tree(metric_column="Avg time/rank", precision=5))

In [ ]:
# print(tkRandom.tree(metric_column="Avg time/rank", precision=5))

In [ ]:
# print(tkReverse.tree(metric_column="Avg time/rank", precision=5))

In [ ]:
# print(tkPerturbed.tree(metric_column="Avg time/rank", precision=5))

In [ ]:
tkRandom.metadata_column_to_perfdata("num_procs")
tkRandom.metadata_column_to_perfdata("input_size")

tkRandom.dataframe = tkRandom.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkSorted.metadata_column_to_perfdata("num_procs")
tkSorted.metadata_column_to_perfdata("input_size")

tkSorted.dataframe = tkSorted.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkReverse.metadata_column_to_perfdata("num_procs")
tkReverse.metadata_column_to_perfdata("input_size")

tkReverse.dataframe = tkReverse.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

tkPerturbed.metadata_column_to_perfdata("num_procs")
tkPerturbed.metadata_column_to_perfdata("input_size")

tkPerturbed.dataframe = tkPerturbed.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()

**4c. Min/Max/Avg/Total/Variance Time per Rank**

In [ ]:
leg = ["MIN", "AVG", "MAX"]
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_mpl(df, df2, df3, title, xlabel, ylabel, input_type):    
    for input_size in df.columns:
        if (input_size != 268435456) and (input_size != 65536):
            continue
        fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o', label='Min')
        ax.plot(processes, df2[input_size], marker='o', label='Avg')
        ax.plot(processes, df3[input_size], marker='o', label='Max')

        ax.set_title(f"{title}, {input_type}, Input size: {input_size}")
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(leg)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        
#         plt.show()
        directory = f"part4_graphs/{input_type}"
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"{directory}/{title}_{input_type}_inputsize_{input_size}.png"
        plt.savefig(filename)
        plt.close(fig)
        
def plot_mplOne(df, title, xlabel, ylabel, typ, input_type):    
    for input_size in df.columns:
        if (input_size != 268435456) and (input_size != 65536):
            continue
        fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, df[input_size], marker='o')
        ax.set_title(f"{title} {typ}, {input_type}, Input size: {input_size}")
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        
#         plt.show()
        directory = f"part4_graphs/{input_type}"
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"{directory}/{title}_{typ}_{input_type}_{input_size}.png"
        plt.savefig(filename)
        plt.close(fig)
        
def plot_mplSpeedup(df, title, xlabel, ylabel, typ, input_type):    
    baseline = df.loc[2];
    for input_size in df.columns:
        if (input_size != 268435456) and (input_size != 65536):
            continue
        fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(processes, baseline[input_size] / df[input_size], marker='o')
        ax.set_title(f"{title} {typ}, {input_type}, Input size: {input_size}")
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        
#         plt.show()
        directory = f"part4_graphs/{input_type}"
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"{directory}/{title}_{typ}_{input_type}_{input_size}.png"
        plt.savefig(filename)
        plt.close(fig)
        
input_types = ['Random', 'Sorted', 'Reverse', 'Perturbed']
thickets = [tkRandom, tkSorted, tkReverse, tkPerturbed]

for tk, input_type in zip(thickets, input_types):
    for node in tk.graph.traverse():
        if node.frame["name"] not in ['main', 'comp_large', 'comm']:
            continue
        
        plot_mpl(
            df=tk.dataframe.loc[node, "Min time/rank"].unstack(level="input_size"),
            df2=tk.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
            df3=tk.dataframe.loc[node, "Max time/rank"].unstack(level="input_size"),
            title=node.frame["name"],
            xlabel="Processes",
            ylabel="Time (seconds)",
            input_type=input_type
        )
        
        plot_mplOne(
            df=tk.dataframe.loc[node, "Total time"].unstack(level="input_size"),
            title=node.frame["name"],
            xlabel="Processes",
            ylabel="Time (seconds)",
            typ="Total Time",
            input_type=input_type
        )
        
        plot_mplOne(
            df=tk.dataframe.loc[node, "Variance time/rank"].unstack(level="input_size"),
            title=node.frame["name"],
            xlabel="Processes",
            ylabel="Time (seconds)",
            typ="Variance",
            input_type=input_type
        )


**For comp_large, comm, main: Strong scaling plots for each input_size with lines for input_type (7 plots - 4 lines each)**

In [ ]:
leg = ["Random", "Sorted", "Reverse", "Perturbed"]
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

def plot_comparison(df_random, df_sorted, df_reverse, df_perturbed, title, xlabel, ylabel):    
    for input_size in df_random.columns:
        fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
        ax = fig.add_subplot(1, 1, 1)
        
        ax.plot(processes, df_random[input_size], marker='o', label='Random')
        ax.plot(processes, df_sorted[input_size], marker='s', label='Sorted')
        ax.plot(processes, df_reverse[input_size], marker='^', label='Reverse')
        ax.plot(processes, df_perturbed[input_size], marker='D', label='Perturbed')
        
        ax.set_title(f"{title}, Input size: {input_size}")
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend()
        plt.xscale("log", base=2)
        plt.xticks(processes)
        ax.set_xticklabels(processes)
        
        directory = f"part5_graphs/strong_scaling"
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"{directory}/{title}_{input_size}.png"
        plt.savefig(filename)
        
#         plt.show()
        plt.close(fig)

for node in tkRandom.graph.traverse():
    if node.frame["name"] not in ['main', 'comp_large', 'comm']:
        continue
    
    node_sorted = next(n for n in tkSorted.graph.traverse() 
                      if n.frame["name"] == node.frame["name"])
    node_reverse = next(n for n in tkReverse.graph.traverse() 
                       if n.frame["name"] == node.frame["name"])
    node_perturbed = next(n for n in tkPerturbed.graph.traverse() 
                         if n.frame["name"] == node.frame["name"])
    
    plot_comparison(
        df_random=tkRandom.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size"),
        df_sorted=tkSorted.dataframe.loc[node_sorted, "Avg time/rank"].unstack(level="input_size"),
        df_reverse=tkReverse.dataframe.loc[node_reverse, "Avg time/rank"].unstack(level="input_size"),
        df_perturbed=tkPerturbed.dataframe.loc[node_perturbed, "Avg time/rank"].unstack(level="input_size"),
        title=node.frame["name"],
        xlabel="Processes",
        ylabel="Time (seconds)",
    )


**Strong scaling speedup plot for each input_type (4 plots)**

In [ ]:
def plot_strong_scaling_speedup(df, input_type, processes, node_name):
    input_sizes = sorted(df.columns)
    fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
    ax = fig.add_subplot(1, 1, 1)
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(input_sizes)))
    
    for i, input_size in enumerate(input_sizes):
        baseline = df[input_size].iloc[0]
        speedup = (baseline / df[input_size]) * 2
        
        ax.plot(processes, speedup, marker='o', label=f'N={input_size}', 
                color=colors[i], linewidth=2, markersize=6)
    
    ax.set_title(f"Strong Scaling Speedup - {input_type} - {node_name}")
    ax.set_xlabel("Processes")
    ax.set_ylabel("Speedup")
    ax.set_xscale('log', base=2)
    ax.set_yscale('log', base=2)
    ax.set_xticks(processes)
    ax.get_xaxis().set_major_formatter(plt.ScalarFormatter())
    ax.legend()
    plt.tight_layout()
    
#     plt.show()
    directory = f"part5_graphs/speedup"
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"{directory}/strong_scaling_{input_type}_{node_name}.png"
    plt.savefig(filename)
    plt.close(fig)

for tk, input_type in zip([tkRandom, tkSorted, tkReverse, tkPerturbed], ['Random', 'Sorted', 'Reverse', 'Perturbed']):
    for node in tk.graph.traverse():
        if node.frame["name"] not in ['main', 'comp_large', 'comm']:
            continue
        node_name = node.frame["name"]
        df = tk.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size")
        df = df.loc[processes]
        
        plot_strong_scaling_speedup(df, input_type, processes, node_name)


**Weak scaling plots for each input_type (4 plots)**

**1. Takes inputs sizes with a constant workload per processor**

In [ ]:
node_names = ['main', 'comp_large', 'comm']

input_types = ['Random', 'Sorted', 'ReverseSorted', '1_perc_perturbed']
thickets = [tkRandom, tkSorted, tkReverse, tkPerturbed]

process_input_pairs = {
    2: 65536,
    8: 262144,
    32: 1048576,
    128: 4194304,
    512: 16777216
}

def plot_weak_scaling(tk, input_type, node_name, process_input_pairs):
    df = tk.dataframe

    try:
        node = next(n for n in tk.graph.traverse() if n.frame["name"] == node_name)
    except StopIteration:
        print(f"Node {node_name} not found in input type {input_type}")
        return

    df_node = df.loc[node, "Avg time/rank"].unstack(level="input_size")
    times = []
    processes_used = []

    for num_procs, input_size in process_input_pairs.items():
        if num_procs not in df_node.index:
            continue
        if input_size not in df_node.columns:
            continue
        time = df_node[input_size].loc[num_procs]
        if pd.isna(time):
            continue
        times.append(time)
        processes_used.append(num_procs)

    if not times:
        print(f"No data available for {node_name} with input type {input_type}")
        return

    fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(processes_used, times, marker='o', label='Weak Scaling')

    ax.set_title(f"Weak Scaling - {input_type} - {node_name}")
    ax.set_xlabel("Processes")
    ax.set_ylabel("Time (seconds)")
    ax.set_xscale('log', base=2)
    ax.set_xticks(processes_used)
    ax.get_xaxis().set_major_formatter(plt.ScalarFormatter())
    ax.legend()
    plt.tight_layout()

#     plt.show()
    directory = f"part5_graphs/weak_scaling"
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"{directory}/weak_scaling_{node_name}_{input_type}.png"
    plt.savefig(filename)
    plt.close(fig)

for tk, input_type in zip([tkRandom, tkSorted, tkReverse, tkPerturbed], ['Random', 'Sorted', 'ReverseSorted', '1_perc_perturbed']):
    for node_name in node_names:
        plot_weak_scaling(tk, input_type, node_name, process_input_pairs)

In [ ]:
sorting_algorithms = ['sample', 'radix', 'bitonic', 'merge']
input_types = ['Random', 'Sorted', 'ReverseSorted', '1_perc_perturbed']
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

thickets = {}

base_dir = 'all_caliper'

for sort_algo in sorting_algorithms:
    thickets[sort_algo] = {}
    for input_type in input_types:
        pattern = os.path.join(base_dir, sort_algo, input_type, '*.cali')
        cali_files = glob(pattern)
        if not cali_files:
            continue
        thicket = th.Thicket.from_caliperreader(cali_files)
        thicket.metadata_column_to_perfdata("num_procs")
        thicket.metadata_column_to_perfdata("input_size")
        thicket.dataframe = thicket.dataframe.reset_index().set_index(["node", "num_procs", "input_size"]).sort_index()
        thickets[sort_algo][input_type] = thicket

input_sizes = [65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456]

node_names = ['comm', 'comp_large', 'main']

def plot_avg_time_rank(thickets, input_type, node_name, input_size, processes):
    fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
    ax = fig.add_subplot(1, 1, 1)
    
    colors = ['blue', 'green', 'red', 'purple']
    markers = ['o', 's', '^', 'D']
    
    sort_algo_names = ['Sample Sort', 'Radix Sort', 'Bitonic Sort', 'Merge Sort']
    
    for idx, sort_algo in enumerate(sorting_algorithms):
        if input_type not in thickets[sort_algo]:
            continue
        thicket = thickets[sort_algo][input_type]
        node = next((n for n in thicket.graph.traverse() if n.frame["name"] == node_name), None)
        if node is None:
            continue
        df = thicket.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size")
        if df.empty:
            continue
        df = df.loc[processes]
        if input_size not in df.columns:
            continue
        times = df[input_size]
        if times.isnull().all():
            continue
        ax.plot(processes, times, marker=markers[idx], color=colors[idx], label=sort_algo_names[idx])
    
    ax.set_title(f"{node_name}, {input_type.replace('_', ' ')}, Input size: {input_size}")
    ax.set_xlabel("Processes")
    ax.set_ylabel("Average Time per Rank (seconds)")
    ax.legend()
    ax.set_xscale("log", base=2)
    ax.set_xticks(processes)
    ax.set_xticklabels(processes)
    ax.grid(True, which="both", ls="-", alpha=0.3)
    
    directory = os.path.join('combined_graphs', input_type, node_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"{node_name}_{input_type}_{input_size}.png"
    filepath = os.path.join(directory, filename)
    plt.savefig(filepath, facecolor=fig.get_facecolor())  # Include facecolor
    plt.close(fig)

def plot_speedup(thickets, input_type, node_name, input_size, processes):
    fig = plt.figure(figsize=(15,7), facecolor=(1, 1, 1))
    ax = fig.add_subplot(1, 1, 1)
    
    colors = ['blue', 'green', 'red', 'purple']
    markers = ['o', 's', '^', 'D']
    
    sort_algo_names = ['Sample Sort', 'Radix Sort', 'Bitonic Sort', 'Merge Sort']
    
    for idx, sort_algo in enumerate(sorting_algorithms):
        if input_type not in thickets[sort_algo]:
            continue
        thicket = thickets[sort_algo][input_type]
        node = next((n for n in thicket.graph.traverse() if n.frame["name"] == node_name), None)
        if node is None:
            continue
        df = thicket.dataframe.loc[node, "Avg time/rank"].unstack(level="input_size")
        if df.empty:
            continue
        df = df.loc[processes]
        if input_size not in df.columns:
            continue
        times = df[input_size]
        if times.isnull().any():
            continue
        if 2 not in times.index:
            continue
        baseline = times.loc[2]
        speedup = (baseline / times) * 2
        ax.plot(processes, speedup, marker=markers[idx], color=colors[idx], label=sort_algo_names[idx])
    
    ax.set_title(f"Speedup - {node_name}, {input_type.replace('_', ' ')}, Input size: {input_size}")
    ax.set_xlabel("Processes")
    ax.set_ylabel("Speedup")
    ax.legend()
    ax.set_xscale("log", base=2)
    ax.set_xticks(processes)
    ax.set_xticklabels(processes)
    
#     plt.show()
    directory = os.path.join('combined_graphs', input_type, 'speedup')
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"{node_name}_speedup_{input_type}_{input_size}.png"
    filepath = os.path.join(directory, filename)
    plt.savefig(filepath, facecolor=fig.get_facecolor())
    plt.close(fig)

for input_type in input_types:
    for input_size in input_sizes:
        for node_name in node_names:
            plot_avg_time_rank(thickets, input_type, node_name, input_size, processes)
            plot_speedup(thickets, input_type, node_name, input_size, processes)


(2/2) Creating Thicket: 100%|██████████| 69/69 [00:02<00:00, 33.42it/s]
